# Qualitätsberichte des Gemeinsamen Bundesausschuss

Diese Notebook gibt eine kleine Einführung wie einzelne Attribute oder auch verschachtelte Attribute aus den XML-Daten ausgelesen werden können.

In [1]:
%load_ext autoreload
%autoreload 2

import re

import pandas as pd

import gba_qualitaetsberichte as qb
from gba_qualitaetsberichte.utils import (replace_and_save_nan, fix_standort, assign_bundesland)

## Personal zur Krankenhaushygiene

Wir extrahieren die Anzahl und die Erläuterungen zu den Hygienepersonal-Indikatoren für Krankenhaushygieniker, Hygienebeauftragte der Ärzte, Hygienebeauftragte in der Pflege und Hygienefachkraefte + noch einige andere nützliche Attribute. Das Ergebnis enthält automatisch Stammdaten zu den Krankenhausstandorten.

Die Abfrage läuft über die Jahre 2014 und 2015 und erlaubt somit Vergleiche zwischen den Jahren.

In [2]:
qr = qb.QualityReports(path='data/', years=[2014, 2015])

hygiene_df = qr.query(dict(
    KHH='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Krankenhaushygieniker/Anzahl',
    KHH_ex='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Krankenhaushygieniker/Erlaeuterungen', 
    HBA='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Hygienebeauftragte_Aerzte/Anzahl',
    HBA_ex='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Hygienebeauftragte_Aerzte/Erlaeuterungen',
    HFK='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Hygienefachkraefte/Anzahl',
    HFK_ex='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Hygienefachkraefte/Erlaeuterungen',
    HBP='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Hygienebeauftragte_Pflege/Anzahl',
    HBP_ex='Umgang_mit_Risiken_in_der_Patientenversorgung/Hygienepersonal/Hygienebeauftragte_Pflege/Erlaeuterungen',
    anzahl_betten='Anzahl_Betten',
    Gesundheits_Krankenpfleger='Personal_des_Krankenhauses/Pflegekraefte/Gesundheits_Krankenpfleger/Anzahl',
    Gesundheits_Kinderkrankenpfleger='Personal_des_Krankenhauses/Pflegekraefte/Gesundheits_Kinderkrankenpfleger/Anzahl',
    Altenpfleger='Personal_des_Krankenhauses/Pflegekraefte/Altenpfleger/Anzahl',
))

hygiene_df = replace_and_save_nan(hygiene_df, ['KHH', 'HBA', 'HFK', 'HBP'])
hygiene_df = fix_standort(hygiene_df)
hygiene_df = assign_bundesland(hygiene_df)

hygiene_df = hygiene_df.set_index(['ik-so', 'year'])
assert hygiene_df.index.is_unique
hygiene_df.head()

[100 %] Time elapsed: 00:01:27 | ETA: 00:00:00
Total time elapsed: 00:01:27


,,index,Altenpfleger,Gesundheits_Kinderkrankenpfleger,Gesundheits_Krankenpfleger,HBA,HBA_ex,HBP,HBP_ex,HFK,HFK_ex,...,HBP_original,so,name,plz,ort,strasse,hausnr,address,ik_bl,bundesland
ik-so,year,,,,,,,,,,,,,,,,,,,,,
260100023-01,2014,0,2.70,45.20,347.40,13.0,None,33.0,None,3.0,None,...,33.0,1.0,Diakonissenkrankenhaus,24939,Flensburg,Knuthstr.,1,Knuthstr. 1,01,Schleswig-Holstein
260100034-00,2014,1,3.00,NaN,223.20,5.0,"Bundesen, Gerold, Koerlings, Matern, Rohlf-Koe...",0.0,None,2.0,"Fr. Hennings, Kautenburger in WB bis 2015",...,0.0,0.0,Malteser Krankenhaus St. Franziskus-Hospital,24939,Flensburg,Waldstraße,17,Waldstraße 17,01,Schleswig-Holstein
260100089-00,2014,2,3.75,57.97,361.37,11.0,None,39.0,None,2.0,None,...,39.0,0.0,Städtisches Krankenhaus Kiel GmbH,24116,Kiel,Chemnitzstr.,33,Chemnitzstr. 33,01,Schleswig-Holstein
260100125-00,2014,3,0.00,1.00,42.33,1.0,None,4.0,None,1.0,None,...,4.0,0.0,Sankt Elisabeth Krankenhaus,24103,Kiel,Königsweg,8,Königsweg 8,01,Schleswig-Holstein
260100147-00,2014,4,NaN,NaN,17.00,1.0,Herr Prof. Dr. med. Volker Kahlke ist Belegarz...,1.0,None,1.0,Die Parkklinik Kiel wird von einer externen Hy...,...,1.0,0.0,Park-Klinik GmbH,24116,Kiel,Goethestrasse,11,Goethestrasse 11,01,Schleswig-Holstein


## Extrahiere Krankenhaushygieniker aus den Erläuterungen

Ein Beispiel, was sich mit den Daten machen lässt. Wir extrahieren die Namen von Personen aus den Erläuterungen der Krankenhaushygieniker und können somit eine Mindestanzahl aufstellen, wie oft gewisse Personen als KHH angegeben wurden.

In [3]:
KHH_NAME = re.compile(r'(?:\W|^)(?:(?:Professor|Herr|Frau|Prof\.|Dr\.|PD|med\.|Hr\.|Fr\.|Priv\.|Dozent)\s+)+(?:[a-z]{2,}\.\s+)*((?:(?:[A-Z][\w-]+|[A-Z]\.|[A-Z\.\-]+)\s+)?([A-Z][\w-]{3,}))')
KHH_NAME.search('''
Zuständig ist Herr Priv. Dozent Dr. Bernhard Jahn-Mühl, Direktor des AGAPLESION  HYGIENE INSTITUTS FÜR HYGIENE UND UMWELTMEDIZIN, Frankfurt am Main. Die Vertretungsregelung ist gewährleistet.
''')

<_sre.SRE_Match object; span=(14, 55), match=' Herr Priv. Dozent Dr. Bernhard Jahn-Mühl'>

In [4]:
def detect_person_names(s):
    if pd.isnull(s):
        return None
    match = KHH_NAME.findall(s)
    return match


def get_khh_list(df):
    for index, row in df.iterrows():
        names = detect_person_names(row['KHH_ex'])
        if not names:
            continue
        for name in names:
            yield {'ik': row['ik'],
                   'index': index,
                   'full_name': name[0],
                  'last_name': name[1],
                   'name_kh': row['name'],
                   'year': index[1]}

In [5]:
khh_df = pd.DataFrame(get_khh_list(hygiene_df))
khh_df.head()

,full_name,ik,index,last_name,name_kh,year
0,Martin Oldenburg,260100023,"(260100023-01, 2014)",Oldenburg,Diakonissenkrankenhaus,2014
1,Martin Oldenburg,260100034,"(260100034-00, 2014)",Oldenburg,Malteser Krankenhaus St. Franziskus-Hospital,2014
2,C. Staehly,260100034,"(260100034-00, 2014)",Staehly,Malteser Krankenhaus St. Franziskus-Hospital,2014
3,Bialek,260100125,"(260100125-00, 2014)",Bialek,Sankt Elisabeth Krankenhaus,2014
4,Christiansen,260100147,"(260100147-00, 2014)",Christiansen,Park-Klinik GmbH,2014


Zeige die Top 10 Personen mit den meisten Erwähnungen an.

In [6]:
khh_multiple = khh_df.groupby(['last_name', 'year']).size()
khh_multiple = khh_multiple.unstack().sort_values(2015, ascending=False)
khh_multiple.head(10)

year,2014,2015
last_name,,
Popp,7.0,14.0
Pitten,4.0,11.0
Schwarzkopf,10.0,11.0
Völker,6.0,8.0
Holfeld,NaN,7.0
Böttcher-Lorenz,NaN,6.0
Parohl,NaN,6.0
Erichsen,NaN,6.0
Tilkes,7.0,6.0


## Personal in den Fachabteilungen

Als zweites Beispiel wollen wir die Fallzahlen und Personalzahlen der Fachabteilungen wissen. Dazu verschachteln wir die Anfrage auf eben des `Organisationseinheit_Fachabteilung` Knotens wie folgt. Pro Ebene kann es nur eine tiefere Verschachtelung geben, um die Ausgabe als Tabelle zu ermöglichen. Für jeden gefundenen, tiefsten Knoten werden alle selektierten Attribute des Eltern-Queries mitausgegeben.

In [8]:
qr = qb.QualityReports(years=[2015])

abteilung_df = qr.query({
    'anzahl_betten_gesamt': './Anzahl_Betten',
    'fallzahl_vs_gesamt': './Fallzahlen/Vollstationaere_Fallzahl',
    'fallzahl_am_gesamt': './Fallzahlen/Ambulante_Fallzahl',
    './/Organisationseinheit_Fachabteilung': {
        'Gliederungsnummer': './Gliederungsnummer',
        'fachabteilung_name': './Name',
        'fallzahl_vs': './Fallzahlen_OE/Vollstationaere_Fallzahl',
        'fallzahl_ts': './Fallzahlen_OE/Teilstationaere_Fallzahl',
        'aerzte_vk': './Personelle_Ausstattung/Aerztliches_Personal/Hauptabteilung/Aerzte_ohne_Belegaerzte/Personalerfassung/Anzahl_VK',
        'fachaerzte_vk': './Personelle_Ausstattung/Aerztliches_Personal/Hauptabteilung/Aerzte_ohne_Belegaerzte/Fachaerzte/Personalerfassung/Anzahl_VK',
        'pflegekraefte_vk': './Personelle_Ausstattung/Pflegekraefte/Gesundheits_Krankenpfleger/Personalerfassung/Anzahl_VK'
    }
})
abteilung_df.head()

[100 %] Time elapsed: 00:00:43 | ETA: 00:00:00
Total time elapsed: 00:00:43


,Gliederungsnummer,aerzte_vk,anzahl_betten_gesamt,fachabteilung_name,fachaerzte_vk,fallzahl_am_gesamt,fallzahl_ts,fallzahl_vs,fallzahl_vs_gesamt,hausnr_kh,...,path_year,pflegekraefte_vk,plz_kh,plz_standort,so_kh,so_standort,strasse_kh,strasse_standort,traeger,traeger_art
0,1.0,35.14,563.0,Medizinische Klinik,15.11,50000.0,123.0,6257.0,27323.0,1,...,2015,93.3,24939.0,24939.0,99.0,1.0,Knuthstr.,Knuthstr.,Ev.-Luth. Diakonissenanstalt zu Flensburg,freigemeinnützig
1,2.0,14.80,563.0,"Klinik für Gynäkologie und Geburtshilfe, Brust...",8.20,50000.0,0.0,4909.0,27323.0,1,...,2015,13.4,24939.0,24939.0,99.0,1.0,Knuthstr.,Knuthstr.,Ev.-Luth. Diakonissenanstalt zu Flensburg,freigemeinnützig
2,3.0,21.50,563.0,"Klinik für Orthopädie, Unfallchirurgie, Handch...",12.40,50000.0,0.0,3503.0,27323.0,1,...,2015,25.9,24939.0,24939.0,99.0,1.0,Knuthstr.,Knuthstr.,Ev.-Luth. Diakonissenanstalt zu Flensburg,freigemeinnützig
3,4.0,15.50,563.0,Kinder- und Jugendmedizin,6.00,50000.0,1.0,2439.0,27323.0,1,...,2015,1.1,24939.0,24939.0,99.0,1.0,Knuthstr.,Knuthstr.,Ev.-Luth. Diakonissenanstalt zu Flensburg,freigemeinnützig
4,5.0,9.61,563.0,Urologische Klinik,6.61,50000.0,1.0,2732.0,27323.0,1,...,2015,22.3,24939.0,24939.0,99.0,1.0,Knuthstr.,Knuthstr.,Ev.-Luth. Diakonissenanstalt zu Flensburg,freigemeinnützig
